In [1]:
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
pd.options.display.max_rows=100
import metrics
from catboost import CatBoostRegressor, Pool
from  collections import Counter
import numpy as np
import itertools
pd.options.mode.chained_assignment = None
import tensorflow
import math
import lightgbm

In [2]:
data_path=Path('data')
users = pd.read_csv(data_path / 'users.tsv', delimiter='\t')
history = pd.read_csv(data_path / 'history.tsv', delimiter='\t')
validate = pd.read_csv(data_path / 'validate.tsv', delimiter='\t')
validate_answers = pd.read_csv(data_path / 'validate_answers.tsv', delimiter='\t')
cpm_probs = pd.read_csv(data_path / 'cpm_probs.tsv', delimiter='\t', index_col='cpm')



In [3]:
# hour_split = validate.hour_start.median()+100
hour_split = validate.hour_start.max()+1

In [4]:
validate_train = validate[validate.hour_start<=hour_split]
validate_val = validate[validate.hour_start>hour_split]
validate_answers_train = validate_answers.loc[validate_train.index]
validate_answers_val = validate_answers.iloc[validate_val.index]
# history_train = history[history.hour<=hour_split]
len(validate_train)

1008

In [5]:
def list_as_ints(l):
    return [int(i) for i in l]
    
def get_users_pub_activity(x, d, f=None):
    s = 0
    for k in itertools.product(list_as_ints(x[0].split(',')), list_as_ints(x[1].split(','))):
        s += d.get(k, 0)
    if f is not None:
        if s > 0:
            s = f(s)
    return s

def get_users_pub_activity_list(x, d, f=None):
    s = {i:0 for i in list_as_ints(x[1].split(','))}
    for k in itertools.product(list_as_ints(x[0].split(',')), list_as_ints(x[1].split(','))):
        s[k[1]] += d.get(k, 0)
    return list(s.values())

def get_users_activity_list(x, d, f=None):
    s = {i:0 for i in list_as_ints(x[1].split(','))}
    for k in list_as_ints(x[1].split(',')):
        s[k] += d.get(k, 0)
    return list(s.values())


activities_range = [31, 75, 1000]

quantilies_ranges = range(30, 101, 5)


users_city = users.city_id.to_dict()
    
for v in [validate_train, validate_val]:
    if len(v) == 0:
        continue
           
    min_hour=v.hour_start.min()
    history_train = history[history.hour<hour_split]

    users_pub_activities = {}
    users_activities = {}
    for i in activities_range:
        users_pub_activities[i] = history_train[history_train.cpm<i].groupby(['publisher', 'user_id'])['hour'].count().to_dict()

        users_activities[i] = history_train[history_train.cpm<i].groupby(['user_id'])['hour'].count().to_dict()



    v['hours_diff'] = v.hour_end - v.hour_start
    
    act_lists = v[['publishers', 'user_ids']].apply(lambda x: get_users_pub_activity_list(x, users_pub_activities[1000]), axis=1)
    for i in quantilies_ranges:
        v['activity_q' + str(i)] = [np.quantile(l, i/100) for l in act_lists]



    act_lists50 = v[['publishers', 'user_ids']].apply(lambda x: get_users_activity_list(x, users_activities[31]), axis=1)

    v['publishers_len'] = v.publishers.map(lambda x: len(x.split(',')))
    v['activity_counter0'] = [len([ll for ll in  l if ll > 0]) for l in act_lists50]
    v['activity_counter10'] = [len([ll for ll in  l if ll > 10]) for l in act_lists50]
    v['activity_counter20'] = [len([ll for ll in  l if ll > 20]) for l in act_lists50]
    v['activity_counter50'] = [len([ll for ll in  l if ll > 50]) for l in act_lists50]
    v['activity_counter150'] = [len([ll for ll in  l if ll > 150]) for l in act_lists50]


    for i in activities_range:
        v['activity_' + str(i)] = v[['publishers', 'user_ids']].apply(lambda x : get_users_pub_activity(x,users_pub_activities[i]), axis=1)
        v['activity_log_' + str(i)] = v[['publishers', 'user_ids']].apply(lambda x : get_users_pub_activity(x,users_pub_activities[i], math.log), axis=1)
        v['activity/audience_size_' + str(i)]= v['activity_' + str(i)]/(v['audience_size']+0.0001)
        v['activity*hours_diff_' + str(i)]= v['activity_' + str(i)]*v['hours_diff']
        v['activity*hours_diff*cpm_' + str(i)]= v['activity_' + str(i)]*v['hours_diff']*v['cpm']
        v['activity*hours_diff*cpm/audience_size_' + str(i)]= v['activity_' + str(i)]*v['hours_diff']*v['cpm']/(v['audience_size']+0.0001)
     

    v_city = validate.user_ids.map(lambda x: Counter([users_city[int(i)] for i in x.split(',')]))
    v['msk_percent'] = v_city.map(lambda x: x[0]/sum(x.values()))
    v['spb_percent'] = v_city.map(lambda x: x[3]/sum(x.values()))
    v['ekb_percent'] = v_city.map(lambda x: x[7]/sum(x.values()))
    v['cities_n'] = v_city.map(lambda x: len(x))


In [6]:
train_columns = ['cpm', 'hours_diff', 'audience_size']

train_columns.append('msk_percent')
train_columns.append('spb_percent')
train_columns.append('ekb_percent')
train_columns.append('cities_n')

train_columns.append('activity_counter10')
train_columns.append('activity_counter20')
train_columns.append('activity_counter50')
train_columns.append('activity_counter150')

for i in activities_range:
    train_columns.append('activity_' + str(i))
    train_columns.append('activity_log_' + str(i))
    train_columns.append('activity/audience_size_' + str(i))
    train_columns.append('activity*hours_diff_' + str(i))
    train_columns.append('activity*hours_diff*cpm_' + str(i)) 
    train_columns.append('activity*hours_diff*cpm/audience_size_' + str(i))

    
for i in quantilies_ranges:
    train_columns.append('activity_q' + str(i))


In [7]:
if len(validate_val)  ==0:
    validate_val = validate_train.copy()
    validate_answers_val = validate_answers_train.copy()

In [8]:
models= {}
for target in ['at_least_one', 'at_least_two', 'at_least_three']:
    train_data = lightgbm.Dataset(validate_train[train_columns].values, label=validate_answers_train[target])
    test_data = lightgbm.Dataset(validate_val[train_columns].values, label=validate_answers_val[target])
    parameters = {
        'application': 'binary',
        'objective': 'cross_entropy',
        'min_data_in_leaf': 2,
        'max_depth':3,
        'num_leaves': 10,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.7,
        'bagging_freq':1,
        'learning_rate': 0.008,
        'verbose': 0,

    }

    model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=train_data,
                           num_boost_round=8000,
                          )

    models[target]=model


[1]	training's cross_entropy: 0.356183
[2]	training's cross_entropy: 0.35473
[3]	training's cross_entropy: 0.353305
[4]	training's cross_entropy: 0.351913
[5]	training's cross_entropy: 0.350561
[6]	training's cross_entropy: 0.349309
[7]	training's cross_entropy: 0.348039
[8]	training's cross_entropy: 0.346811
[9]	training's cross_entropy: 0.345609
[10]	training's cross_entropy: 0.3445
[11]	training's cross_entropy: 0.343327
[12]	training's cross_entropy: 0.342206
[13]	training's cross_entropy: 0.341093
[14]	training's cross_entropy: 0.34001
[15]	training's cross_entropy: 0.338941
[16]	training's cross_entropy: 0.337926
[17]	training's cross_entropy: 0.336927
[18]	training's cross_entropy: 0.335946
[19]	training's cross_entropy: 0.334969
[20]	training's cross_entropy: 0.334031
[21]	training's cross_entropy: 0.333095
[22]	training's cross_entropy: 0.332229
[23]	training's cross_entropy: 0.331339
[24]	training's cross_entropy: 0.330468
[25]	training's cross_entropy: 0.329606
[26]	training

[376]	training's cross_entropy: 0.271999
[377]	training's cross_entropy: 0.271986
[378]	training's cross_entropy: 0.271973
[379]	training's cross_entropy: 0.271961
[380]	training's cross_entropy: 0.271947
[381]	training's cross_entropy: 0.271934
[382]	training's cross_entropy: 0.271921
[383]	training's cross_entropy: 0.271909
[384]	training's cross_entropy: 0.271896
[385]	training's cross_entropy: 0.271884
[386]	training's cross_entropy: 0.271873
[387]	training's cross_entropy: 0.271862
[388]	training's cross_entropy: 0.27185
[389]	training's cross_entropy: 0.271838
[390]	training's cross_entropy: 0.271827
[391]	training's cross_entropy: 0.271816
[392]	training's cross_entropy: 0.271803
[393]	training's cross_entropy: 0.271791
[394]	training's cross_entropy: 0.27178
[395]	training's cross_entropy: 0.271768
[396]	training's cross_entropy: 0.271758
[397]	training's cross_entropy: 0.271746
[398]	training's cross_entropy: 0.271736
[399]	training's cross_entropy: 0.271726
[400]	training's c

[829]	training's cross_entropy: 0.270228
[830]	training's cross_entropy: 0.270227
[831]	training's cross_entropy: 0.270226
[832]	training's cross_entropy: 0.270226
[833]	training's cross_entropy: 0.270225
[834]	training's cross_entropy: 0.270223
[835]	training's cross_entropy: 0.270222
[836]	training's cross_entropy: 0.270221
[837]	training's cross_entropy: 0.27022
[838]	training's cross_entropy: 0.270219
[839]	training's cross_entropy: 0.270217
[840]	training's cross_entropy: 0.270215
[841]	training's cross_entropy: 0.270214
[842]	training's cross_entropy: 0.270213
[843]	training's cross_entropy: 0.270212
[844]	training's cross_entropy: 0.270211
[845]	training's cross_entropy: 0.270209
[846]	training's cross_entropy: 0.270208
[847]	training's cross_entropy: 0.270206
[848]	training's cross_entropy: 0.270205
[849]	training's cross_entropy: 0.270203
[850]	training's cross_entropy: 0.270202
[851]	training's cross_entropy: 0.270202
[852]	training's cross_entropy: 0.270201
[853]	training's 

[1370]	training's cross_entropy: 0.269839
[1371]	training's cross_entropy: 0.269839
[1372]	training's cross_entropy: 0.269838
[1373]	training's cross_entropy: 0.269838
[1374]	training's cross_entropy: 0.269837
[1375]	training's cross_entropy: 0.269837
[1376]	training's cross_entropy: 0.269836
[1377]	training's cross_entropy: 0.269836
[1378]	training's cross_entropy: 0.269835
[1379]	training's cross_entropy: 0.269835
[1380]	training's cross_entropy: 0.269834
[1381]	training's cross_entropy: 0.269834
[1382]	training's cross_entropy: 0.269834
[1383]	training's cross_entropy: 0.269833
[1384]	training's cross_entropy: 0.269833
[1385]	training's cross_entropy: 0.269832
[1386]	training's cross_entropy: 0.269832
[1387]	training's cross_entropy: 0.269831
[1388]	training's cross_entropy: 0.269831
[1389]	training's cross_entropy: 0.269831
[1390]	training's cross_entropy: 0.26983
[1391]	training's cross_entropy: 0.269829
[1392]	training's cross_entropy: 0.269829
[1393]	training's cross_entropy: 0.

[1935]	training's cross_entropy: 0.269623
[1936]	training's cross_entropy: 0.269623
[1937]	training's cross_entropy: 0.269623
[1938]	training's cross_entropy: 0.269622
[1939]	training's cross_entropy: 0.269622
[1940]	training's cross_entropy: 0.269622
[1941]	training's cross_entropy: 0.269621
[1942]	training's cross_entropy: 0.269621
[1943]	training's cross_entropy: 0.269621
[1944]	training's cross_entropy: 0.26962
[1945]	training's cross_entropy: 0.26962
[1946]	training's cross_entropy: 0.26962
[1947]	training's cross_entropy: 0.269619
[1948]	training's cross_entropy: 0.269619
[1949]	training's cross_entropy: 0.269619
[1950]	training's cross_entropy: 0.269618
[1951]	training's cross_entropy: 0.269618
[1952]	training's cross_entropy: 0.269618
[1953]	training's cross_entropy: 0.269617
[1954]	training's cross_entropy: 0.269617
[1955]	training's cross_entropy: 0.269616
[1956]	training's cross_entropy: 0.269616
[1957]	training's cross_entropy: 0.269616
[1958]	training's cross_entropy: 0.26

...

[5938]	training's cross_entropy: 0.12062
[5939]	training's cross_entropy: 0.12062
[5940]	training's cross_entropy: 0.12062
[5941]	training's cross_entropy: 0.12062
[5942]	training's cross_entropy: 0.12062
[5943]	training's cross_entropy: 0.12062
[5944]	training's cross_entropy: 0.12062
[5945]	training's cross_entropy: 0.12062
[5946]	training's cross_entropy: 0.12062
[5947]	training's cross_entropy: 0.12062
[5948]	training's cross_entropy: 0.12062
[5949]	training's cross_entropy: 0.12062
[5950]	training's cross_entropy: 0.12062
[5951]	training's cross_entropy: 0.12062
[5952]	training's cross_entropy: 0.12062
[5953]	training's cross_entropy: 0.12062
[5954]	training's cross_entropy: 0.12062
[5955]	training's cross_entropy: 0.12062
[5956]	training's cross_entropy: 0.12062
[5957]	training's cross_entropy: 0.12062
[5958]	training's cross_entropy: 0.12062
[5959]	training's cross_entropy: 0.12062
[5960]	training's cross_entropy: 0.12062
[5961]	training's cross_entropy: 0.12062
[5962]	training'

[6521]	training's cross_entropy: 0.120615
[6522]	training's cross_entropy: 0.120615
[6523]	training's cross_entropy: 0.120615
[6524]	training's cross_entropy: 0.120615
[6525]	training's cross_entropy: 0.120615
[6526]	training's cross_entropy: 0.120615
[6527]	training's cross_entropy: 0.120615
[6528]	training's cross_entropy: 0.120615
[6529]	training's cross_entropy: 0.120615
[6530]	training's cross_entropy: 0.120615
[6531]	training's cross_entropy: 0.120615
[6532]	training's cross_entropy: 0.120615
[6533]	training's cross_entropy: 0.120615
[6534]	training's cross_entropy: 0.120615
[6535]	training's cross_entropy: 0.120615
[6536]	training's cross_entropy: 0.120615
[6537]	training's cross_entropy: 0.120615
[6538]	training's cross_entropy: 0.120615
[6539]	training's cross_entropy: 0.120615
[6540]	training's cross_entropy: 0.120615
[6541]	training's cross_entropy: 0.120615
[6542]	training's cross_entropy: 0.120615
[6543]	training's cross_entropy: 0.120615
[6544]	training's cross_entropy: 0

[7101]	training's cross_entropy: 0.120611
[7102]	training's cross_entropy: 0.120611
[7103]	training's cross_entropy: 0.120611
[7104]	training's cross_entropy: 0.120611
[7105]	training's cross_entropy: 0.120611
[7106]	training's cross_entropy: 0.120611
[7107]	training's cross_entropy: 0.120611
[7108]	training's cross_entropy: 0.120611
[7109]	training's cross_entropy: 0.120611
[7110]	training's cross_entropy: 0.120611
[7111]	training's cross_entropy: 0.120611
[7112]	training's cross_entropy: 0.120611
[7113]	training's cross_entropy: 0.120611
[7114]	training's cross_entropy: 0.120611
[7115]	training's cross_entropy: 0.120611
[7116]	training's cross_entropy: 0.120611
[7117]	training's cross_entropy: 0.120611
[7118]	training's cross_entropy: 0.120611
[7119]	training's cross_entropy: 0.120611
[7120]	training's cross_entropy: 0.120611
[7121]	training's cross_entropy: 0.120611
[7122]	training's cross_entropy: 0.120611
[7123]	training's cross_entropy: 0.120611
[7124]	training's cross_entropy: 0

[7530]	training's cross_entropy: 0.120609
[7531]	training's cross_entropy: 0.120609
[7532]	training's cross_entropy: 0.120609
[7533]	training's cross_entropy: 0.120609
[7534]	training's cross_entropy: 0.120609
[7535]	training's cross_entropy: 0.120609
[7536]	training's cross_entropy: 0.120609
[7537]	training's cross_entropy: 0.120609
[7538]	training's cross_entropy: 0.120609
[7539]	training's cross_entropy: 0.120609
[7540]	training's cross_entropy: 0.120609
[7541]	training's cross_entropy: 0.120609
[7542]	training's cross_entropy: 0.120609
[7543]	training's cross_entropy: 0.120609
[7544]	training's cross_entropy: 0.120609
[7545]	training's cross_entropy: 0.120609
[7546]	training's cross_entropy: 0.120609
[7547]	training's cross_entropy: 0.120609
[7548]	training's cross_entropy: 0.120609
[7549]	training's cross_entropy: 0.120609
[7550]	training's cross_entropy: 0.120609
[7551]	training's cross_entropy: 0.120609
[7552]	training's cross_entropy: 0.120609
[7553]	training's cross_entropy: 0

[7975]	training's cross_entropy: 0.120607
[7976]	training's cross_entropy: 0.120607
[7977]	training's cross_entropy: 0.120607
[7978]	training's cross_entropy: 0.120607
[7979]	training's cross_entropy: 0.120607
[7980]	training's cross_entropy: 0.120607
[7981]	training's cross_entropy: 0.120607
[7982]	training's cross_entropy: 0.120607
[7983]	training's cross_entropy: 0.120607
[7984]	training's cross_entropy: 0.120607
[7985]	training's cross_entropy: 0.120607
[7986]	training's cross_entropy: 0.120607
[7987]	training's cross_entropy: 0.120607
[7988]	training's cross_entropy: 0.120607
[7989]	training's cross_entropy: 0.120607
[7990]	training's cross_entropy: 0.120607
[7991]	training's cross_entropy: 0.120607
[7992]	training's cross_entropy: 0.120607
[7993]	training's cross_entropy: 0.120607
[7994]	training's cross_entropy: 0.120607
[7995]	training's cross_entropy: 0.120607
[7996]	training's cross_entropy: 0.120607
[7997]	training's cross_entropy: 0.120607
[7998]	training's cross_entropy: 0

In [9]:
validate_val.loc[:,'at_least_one']=models['at_least_one'].predict(validate_val[train_columns].values)
validate_val.loc[:,'at_least_two']=models['at_least_two'].predict(validate_val[train_columns].values)
validate_val.loc[:,'at_least_three']=models['at_least_three'].predict(validate_val[train_columns].values)
print(metrics.get_smoothed_mean_log_accuracy_ratio(validate_answers_val, validate_val))
  

2.34


In [11]:
np.save('models_v1', models)

In [21]:
ms = np.load('models_v1.npy', allow_pickle=True).item()

In [22]:
pred_m = pd.read_csv('model2.csv', index_col='i')
print(metrics.get_smoothed_mean_log_accuracy_ratio(validate_answers_val, pred_m))
for i in [2]:
    print(i)
    pred_m['at_least_one'] += models['at_least_one'].predict(validate_val[train_columns].values)*0.7
    pred_m['at_least_two'] += models['at_least_two'].predict(validate_val[train_columns].values)*0.5
    pred_m['at_least_three'] += models['at_least_three'].predict(validate_val[train_columns].values)*0.5
pred_m['at_least_one']/=1.7
pred_m['at_least_two']/=1.5
pred_m['at_least_three']/=1.5
print(metrics.get_smoothed_mean_log_accuracy_ratio(validate_answers_val, pred_m))

12.85
2
12.43


In [23]:
pd.DataFrame({'Value':models['at_least_one'].feature_importance(),'Feature':train_columns})

,Value,Feature
0,4967,cpm
1,3884,hours_diff
2,2997,audience_size
3,2785,msk_percent
4,2865,spb_percent
5,3206,ekb_percent
6,2440,cities_n
7,2008,activity_counter10
8,1447,activity_counter20
9,1629,activity_counter50


In [24]:
pred_m.at_least_one

i
1       0.014869
2       0.090622
3       0.255002
5       0.020075
6       0.397195
          ...   
985     0.002697
990     0.076250
994     0.047960
1001    0.045055
1002    0.059127
Name: at_least_one, Length: 342, dtype: float64